# Luminance Inverter

Bright screens when you're trying to sleep are no fun.  I really like the program [f.lux](https://justgetflux.com/) for reducing the amount of blue light that comes from the display, but reading a white-background webpage at night can still be unpleasant.

You could always do a straight inversion, but then the colors aren't right.  I got to thinking about colorspaces and wondered what things would look like if you kept the same hue and saturation, but inverted the brightness or luminance.  You'd maintain a similar theme and contrast, but it would tone things down a bit if they were too bright.  (And, conversely, if they were too dark, make them real bright)

This little workbook is me figuring out what that actually looks like.

Put any images you want converted in the `images` directory.  This will find them, invert the luminance, and drop the result in the `inverted` directory.

**Warning** This thing is slow.  Several seconds per image.

Images are expected to be color images in sRGB.

Developed on Python 3 with Anaconda, but so long as you have skimage and juptyer you should be ok.  LMK if I'm wrong.


In [163]:
   # Savagely copy-pasted from https://github.com/alexanderkuk/log-progress 
   def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        ) 

In [173]:
from skimage import io
import colorsys
import glob
import os

if not os.path.exists('inverted'):
    os.makedirs('inverted')

filenames = glob.glob('images/*')

for filename in log_progress(filenames, name="Input Files"):
    img = io.imread(filename)
    
    total_lines = float(len(img))
    for line in log_progress(img, name="{} rows processed".format(filename)):
        for pixel in line:
            r = pixel[0]
            g = pixel[1]
            b = pixel[2]

            h, l, s = colorsys.rgb_to_hls(r/255., g/255., b/255.)

            l = (1. - l) * 0.95

            r, g, b = colorsys.hls_to_rgb(h, l, s)
            pixel[0] = int(r * 255)
            pixel[1] = int(g * 255)
            pixel[2] = int(b * 255)

    io.imsave(os.path.join("inverted", os.path.split(filename)[-1]), img)


